## Modelling and Deployment using MLOps

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import sys
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
import wave
from jiwer import wer

In [2]:
# loading scripts
sys.path.insert(1, '../scripts')
sys.path.append("..")
sys.path.append(".")

from data_cleaning import DataCleaner
from data_viz import Data_Viz

DC = DataCleaner("../logs/preprocessing_notebook.log")
DV = Data_Viz()

In [3]:
# loading datasets
train_meta = DC.meta_loader("../data/train_meta.csv", "csv")
valid_meta = DC.meta_loader("../data/test_meta.csv", "csv")

print(f"Size of the training set: {len(train_meta)}")
print(f"Size of the validation set: {len(valid_meta)}")


Size of the training set: 797
Size of the validation set: 199


In [4]:
train_meta["Target"][0]

'በ ናዝሬት ከተማ የ ሚገኘው የ ሞያ ና ቴክኒክ ኮሌጅ ተማሪዎች ካለፈ ው ሰኞ ጀምሮ አድማ በ መምታት ትምህርት ማቋረጣ ቸውን ከ ስፍራው የደረሰ ን ዘገባ ያስረዳ ል'

### Preprocessing | Prepare the vocabulary

In [5]:
# The set of characters accepted in the transcription.
characters = """
ሀ ሁ ሂ ሄ ህ ሆ
ለ ሉ ሊ ላ ሌ ል ሎ ሏ
መ ሙ ሚ ማ ሜ ም ሞ ሟ
ረ ሩ ሪ ራ ሬ ር ሮ ሯ
ሰ ሱ ሲ ሳ ሴ ስ ሶ ሷ
ሸ ሹ ሺ ሻ ሼ ሽ ሾ ሿ
ቀ ቁ ቂ ቃ ቄ ቅ ቆ ቋ
በ ቡ ቢ ባ ቤ ብ ቦ ቧ
ቨ ቩ ቪ ቫ ቬ ቭ ቮ ቯ
ተ ቱ ቲ ታ ቴ ት ቶ ቷ
ቸ ቹ ቺ ቻ ቼ ች ቾ ቿ
ኋ
ነ ኑ ኒ ና ኔ ን ኖ ኗ
ኘ ኙ ኚ ኛ ኜ ኝ ኞ ኟ
አ ኡ ኢ ኤ እ ኦ
ኧ
ከ ኩ ኪ ካ ኬ ክ ኮ
ኳ
ወ ዉ ዊ ዋ ዌ ው ዎ
ዘ ዙ ዚ ዛ ዜ ዝ ዞ ዟ
ዠ ዡ ዢ ዣ ዤ ዥ ዦ ዧ
የ ዩ ዪ ያ ዬ ይ ዮ
ደ ዱ ዲ ዳ ዴ ድ ዶ ዷ
ጀ ጁ ጂ ጃ ጄ ጅ ጆ ጇ
ገ ጉ ጊ ጋ ጌ ግ ጐ ጓ ጔ
ጠ ጡ ጢ ጣ ጤ ጥ ጦ ጧ
ጨ ጩ ጪ ጫ ጬ ጭ ጮ ጯ
ጰ ጱ ጲ ጳ ጴ ጵ ጶ ጷ
ፀ ፁ ፂ ፃ ፄ ፅ ፆ ፇ
ፈ ፉ ፊ ፋ ፌ ፍ ፎ ፏ
ፐ ፑ ፒ ፓ ፔ ፕ ፖ
""".replace('\n',' ').split(' ')
characters = characters[:-1]
characters.insert(1,' ')

In [6]:
# Mapping characters to integers
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")
# Mapping integers back to original characters
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

The vocabulary is: ['', ' ', 'ሀ', 'ሁ', 'ሂ', 'ሄ', 'ህ', 'ሆ', 'ለ', 'ሉ', 'ሊ', 'ላ', 'ሌ', 'ል', 'ሎ', 'ሏ', 'መ', 'ሙ', 'ሚ', 'ማ', 'ሜ', 'ም', 'ሞ', 'ሟ', 'ረ', 'ሩ', 'ሪ', 'ራ', 'ሬ', 'ር', 'ሮ', 'ሯ', 'ሰ', 'ሱ', 'ሲ', 'ሳ', 'ሴ', 'ስ', 'ሶ', 'ሷ', 'ሸ', 'ሹ', 'ሺ', 'ሻ', 'ሼ', 'ሽ', 'ሾ', 'ሿ', 'ቀ', 'ቁ', 'ቂ', 'ቃ', 'ቄ', 'ቅ', 'ቆ', 'ቋ', 'በ', 'ቡ', 'ቢ', 'ባ', 'ቤ', 'ብ', 'ቦ', 'ቧ', 'ቨ', 'ቩ', 'ቪ', 'ቫ', 'ቬ', 'ቭ', 'ቮ', 'ቯ', 'ተ', 'ቱ', 'ቲ', 'ታ', 'ቴ', 'ት', 'ቶ', 'ቷ', 'ቸ', 'ቹ', 'ቺ', 'ቻ', 'ቼ', 'ች', 'ቾ', 'ቿ', 'ኋ', 'ነ', 'ኑ', 'ኒ', 'ና', 'ኔ', 'ን', 'ኖ', 'ኗ', 'ኘ', 'ኙ', 'ኚ', 'ኛ', 'ኜ', 'ኝ', 'ኞ', 'ኟ', 'አ', 'ኡ', 'ኢ', 'ኤ', 'እ', 'ኦ', 'ኧ', 'ከ', 'ኩ', 'ኪ', 'ካ', 'ኬ', 'ክ', 'ኮ', 'ኳ', 'ወ', 'ዉ', 'ዊ', 'ዋ', 'ዌ', 'ው', 'ዎ', 'ዘ', 'ዙ', 'ዚ', 'ዛ', 'ዜ', 'ዝ', 'ዞ', 'ዟ', 'ዠ', 'ዡ', 'ዢ', 'ዣ', 'ዤ', 'ዥ', 'ዦ', 'ዧ', 'የ', 'ዩ', 'ዪ', 'ያ', 'ዬ', 'ይ', 'ዮ', 'ደ', 'ዱ', 'ዲ', 'ዳ', 'ዴ', 'ድ', 'ዶ', 'ዷ', 'ጀ', 'ጁ', 'ጂ', 'ጃ', 'ጄ', 'ጅ', 'ጆ', 'ጇ', 'ገ', 'ጉ', 'ጊ', 'ጋ', 'ጌ', 'ግ', 'ጐ', 'ጓ', 'ጔ', 'ጠ', 'ጡ', 'ጢ', 'ጣ', 'ጤ', 'ጥ', 'ጦ', 'ጧ', 'ጨ', 'ጩ', 'ጪ', 'ጫ', 'ጬ', 'ጭ', 'ጮ', 'ጯ', 'ጰ', 'ጱ', 'ጲ', 'ጳ', 'ጴ', '

Next is to create function that describes the transformation to apply to each element

In [7]:
train_meta.head(3)

,Unnamed: 0,Target,Feature,Output,Duration,n_channel
0,0,በ ናዝሬት ከተማ የ ሚገኘው የ ሞያ ና ቴክኒክ ኮሌጅ ተማሪዎች ካለፈ ው ...,../data/train/wav/tr_1026_tr11027.wav,../data/train_new/tr_1026_tr11027.wav,12.16,2
1,1,ዘመናዊ ውን የ አምሪ ካን ሂ ዎት ሲ ቀጭ ከርሞ ጰጰሰ ይ ሉናል,../data/train/wav/tr_10603_tr04117.wav,../data/train_new/tr_10603_tr04117.wav,5.12,2
2,2,እ ብዱ አስፋልቱ ላይ የ ኰለኰ ለ ው ድንጋይ መኪና አላ ሳልፍ አለ,../data/train/wav/tr_10521_tr53129.wav,../data/train_new/tr_10521_tr53129.wav,4.48,2


In [8]:
train_meta["Output"]

0        ../data/train_new/tr_1026_tr11027.wav
1       ../data/train_new/tr_10603_tr04117.wav
2       ../data/train_new/tr_10521_tr53129.wav
3       ../data/train_new/tr_10633_tr04147.wav
4      ../data/train_new/tr_10245_tr099087.wav
                        ...                   
792    ../data/train_new/tr_10170_tr099012.wav
793    ../data/train_new/tr_10295_tr100017.wav
794    ../data/train_new/tr_10193_tr099035.wav
795    ../data/train_new/tr_10301_tr100023.wav
796     ../data/train_new/tr_10537_tr53145.wav
Name: Output, Length: 797, dtype: object

In [9]:
wav_file = train_meta["Feature"][0]
wav_file

'../data/train/wav/tr_1026_tr11027.wav'

In [10]:
# An integer scalar Tensor. The window length in samples.
frame_length = 256 #this should be less than or equal to fft length
# An integer scalar Tensor. The number of samples to step.
frame_step = 160
# An integer scalar Tensor. The size of the FFT to apply.
# If not provided, uses the smallest power of 2 enclosing frame_length.
fft_length = 256
# data =  wave.open('../data/train_new/tr_10707_tr03124.wav')
def encode_single_sample(wav_file, label):
    """
    Process the Audio
    
    """
    #read wav file
    file = tf.io.read_file(wav_file)
    #decode voice file
    audio, _ =tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)
    #change type to float32
    audio = tf.cast(audio, tf.float32)    
    #get the spectrogram
    spectrogram = tf.signal.stft(audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length)
    #we only need the magnitude of the spectrogram
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)
    #normalization
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs= tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)
    """
    Process the label
    """
    #convert label to lower case
    label = tf.strings.lower(label)
    #split label
    label = tf.strings.unicode_split(label, input_encoding="UTF-8")
    # Map the characters in label to numbers
    label = char_to_num(label)
    #  Return a dict as our model is expecting two inputs
    return spectrogram, label


In [11]:
valid_meta.head(3)

,Unnamed: 0,Target,Feature,Output,Duration,n_channel
0,797,ካድሬው እዚህ ነው ያለው ማእከላዊ ኮሚቴው ም እዚህ ይገኛል,../data/train/wav/tr_1111_tr12012.wav,../data/train_new/tr_1111_tr12012.wav,5.632,2
1,798,አንበሳው ስጋ ስላ የ አጉረመረ መ,../data/train/wav/tr_10782_tr02105.wav,../data/train_new/tr_10782_tr02105.wav,2.944,2
2,799,ድህነት የማ ያልፍ መስሏቸው ም ንም ሳት በድ ለ ቸው በ ስድብ ሞለጯት ይ...,../data/train/wav/tr_10694_tr03111.wav,../data/train_new/tr_10694_tr03111.wav,9.216,2


### Creating dataset Objects

In [12]:
batch_size = 22
# Define the trainig dataset
train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(train_meta["Feature"]), list(train_meta["Target"]))
)
train_dataset = (
    train_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

# Define the validation dataset
validation_dataset = tf.data.Dataset.from_tensor_slices(
    (list(valid_meta["Feature"]), list(valid_meta["Target"]))
)
validation_dataset = (
    validation_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

In [13]:
print(train_dataset)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 129), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>


In [14]:
print(validation_dataset)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 129), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>


### Model Building 

In [15]:
# defining CTC loss function
def CTCLoss(y_true, y_pred):
    #compute the training-time loss value 
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [16]:
# defining our model
def build_model(input_dim, output_dim, rnn_layers = 5, rnn_units =128):
    #Model input
    input_spectrogram = keras.layers.Input(shape=(None, input_dim), name="input")
    #Expand the dimensions to use 2D CNN
    x = layers.Reshape((-1, input_dim, 1), name="expand_dim")(input_spectrogram)
    #convolutions layer 1
    x = layers.Conv2D(filters=32, 
                      kernel_size=(11, 41), 
                      padding="same", 
                      strides = [2,2],
                      activation="relu", 
                      name="conv_1")(x)
    x = layers.BatchNormalization(name="conv_1_bn")(x)
    x = layers.ReLU(name="conv_1_relu")(x)
    #convolution layer 2
    x = layers.Conv2D(
        filters=32,
        kernel_size=[11, 21],
        strides=[1, 2],
        padding="same",
        activation = "relu",
        use_bias=False,
        name="conv_2",
    )(x)
    x = layers.BatchNormalization(name="conv_2_bn")(x)
    x = layers.ReLU(name="conv_2_relu")(x)
    # Reshape the resulted volume to feed the RNNs layers
    x = layers.Reshape((-1, x.shape[-2] * x.shape[-1]))(x)
    # RNN layers
    for i in range(1, rnn_layers + 1):
        recurrent = layers.GRU(
            units=rnn_units,
            activation="tanh",
            recurrent_activation="sigmoid",
            use_bias=True,
            return_sequences=True,
            reset_after=True,
            name=f"gru_{i}",
        )
        x = layers.Bidirectional(
            recurrent, name=f"bidirectional_{i}", merge_mode="concat"
        )(x)
        if i < rnn_layers:
            x = layers.Dropout(rate=0.5)(x)
    # Dense layer
    x = layers.Dense(units=rnn_units * 2, name="dense_1")(x)
    x = layers.ReLU(name="dense_1_relu")(x)
    x = layers.Dropout(rate=0.5)(x)
    # Classification layer
    output = layers.Dense(units=output_dim + 1, activation="softmax")(x)
    # Model
    model = keras.Model(input_spectrogram, output, name="DeepSpeech_2")
    # Optimizer
    opt = keras.optimizers.Adam(learning_rate=1e-4)
    # Compile the model and return
    model.compile(optimizer=opt, loss=CTCLoss)
    return model


# Get the model
model = build_model(
    input_dim=fft_length // 2 + 1,
    output_dim=char_to_num.vocabulary_size(),
    rnn_units=5,
)
model.summary(line_length=110)


Model: "DeepSpeech_2"
______________________________________________________________________________________________________________
 Layer (type)                                    Output Shape                                Param #          
 input (InputLayer)                              [(None, None, 129)]                         0                
                                                                                                              
 expand_dim (Reshape)                            (None, None, 129, 1)                        0                
                                                                                                              
 conv_1 (Conv2D)                                 (None, None, 65, 32)                        14464            
                                                                                                              
 conv_1_bn (BatchNormalization)                  (None, None, 65, 32)                     

### Training and Evaluating the model

In [17]:
# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    # Iterate over the results and get back the text
    output_text = []
    for result in results:
        result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
        output_text.append(result)
    return output_text


# A callback class to output a few transcriptions during training
class CallbackEval(keras.callbacks.Callback):
    """Displays a batch of outputs after every epoch."""

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def on_epoch_end(self, epoch: int, logs=None):
        predictions = []
        targets = []
        for batch in self.dataset:
            X, y = batch
            batch_predictions = model.predict(X)
            batch_predictions = decode_batch_predictions(batch_predictions)
            predictions.extend(batch_predictions)
            for label in y:
                label = (
                    tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
                )
                targets.append(label)
        wer_score = wer(targets, predictions)
        print("-" * 100)
        print(f"Word Error Rate: {wer_score:.4f}")
        print("-" * 100)
        for i in np.random.randint(0, len(predictions), 2):
            print(f"Target    : {targets[i]}")
            print(f"Prediction: {predictions[i]}")
            print("-" * 100)

In [18]:
#defining the number of epochs
epochs = 3
#callback function to check transcriptions
validation_callback = CallbackEval(validation_dataset)
#Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    callbacks=[validation_callback],
)

Epoch 1/3
1/1 [==============================] - 0s 348ms/step
----------------------------------------------------------------------------------------------------
Word Error Rate: 0.9728
----------------------------------------------------------------------------------------------------
Target    : የተፈራ አተር ቀድሞ ስለ ተዘራ ቶሎ ጐመራ
Prediction: ሁቦሜፅፑሜፅፑፅፑፅ  ን
----------------------------------------------------------------------------------------------------
Target    : ይህንን ም የ አቶ ወንድሙ ን አባባል ለ ማረጋገጥ አንዳንድ ምእራባውያን ጋዜጠኞች ኢትዮጵያውያኑ ይገኙ ባቸዋል ተብሎ በሚ ገመቱ ባቸው የ አስመራ ገበያ ዎች ውስጥ ተዘዋው ረው ለ ማየት ሞክረ ዋል
Prediction: ሁቦሜፅሜፅሜፅሜፅሜፅቦሜፅሜፅሜፅሜፅሜፅሜፅሜፅሜፅፑ
----------------------------------------------------------------------------------------------------
37/37 [==============================] - 1236s 31s/step - loss: 2801.4585 - val_loss: 2944.4263
Epoch 2/3
1/1 [==============================] - 0s 233ms/step
----------------------------------------------------------------------------------------------------
Word

### Inference

In [19]:
# Let's check results on more validation samples
predictions = []
targets = []
for batch in validation_dataset:
    X, y = batch
    batch_predictions = model.predict(X)
    batch_predictions = decode_batch_predictions(batch_predictions)
    predictions.extend(batch_predictions)
    for label in y:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        targets.append(label)
wer_score = wer(targets, predictions)
print("-" * 100)
print(f"Word Error Rate: {wer_score:.4f}")
print("-" * 100)
for i in np.random.randint(0, len(predictions), 5):
    print(f"Target    : {targets[i]}")
    print(f"Prediction: {predictions[i]}")
    print("-" * 100)

1/1 [==============================] - 0s 297ms/step
----------------------------------------------------------------------------------------------------
Word Error Rate: 0.9992
----------------------------------------------------------------------------------------------------
Target    : ኢትዮጵያ ክርስትና የ ተቀበለችው በዛ ጔ ስርወ መንግስት ነው
Prediction: መፅመ 
----------------------------------------------------------------------------------------------------
Target    : በተለይ በ ተረጋጋ ሁኔታ አንቶ ኖ ቩ ድብደባ ሲ ያካሂዱ በ ፊልም መቅረ ና መረጋጋቱ ን የሚ ያሳየው ደግሞ የ ፊልሙ ጥራት መሆኑን እንዳስ ደሰታቸው ብዙዎቹ ገልዋል
Prediction: መፅመፅመፅ 
----------------------------------------------------------------------------------------------------
Target    : አብደላ እድሪስ ወደ ግብ እንዳይ ገቡ ታገዱ
Prediction: መ 
----------------------------------------------------------------------------------------------------
Target    : ት ግስት እህቷ ን ስት ጓጉ ክረ ሚ እንጂ የኔ ን አይነት ሀብል አይገዛ ልሽ ም አለ ቻት
Prediction: መፅመ 
-------------------------------------------------------------------------

In [20]:
pickle.dump(model, open('model.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://3de03d2f-e07a-4342-aacc-afcea6c5c8f0/assets


INFO:tensorflow:Assets written to: ram://3de03d2f-e07a-4342-aacc-afcea6c5c8f0/assets
